In [1]:
import keras 
import tensorflow as tf
from keras.layers import Layer 
from keras import Model 
from keras.losses import SparseCategoricalCrossentropy
import tensorflow_datasets as tfds

c:\Users\hosma\anaconda3\envs\machineLearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data Process

In [2]:
(train_set,test_set),ds_info = tfds.load('mnist',with_info=True, split=['train',"test"], shuffle_files=True,as_supervised=True)

In [3]:
def normalize_image(image,label):
    
    image = tf.reshape(image,[image.shape[0]*image.shape[1]])
 
    return tf.cast(image,tf.float32)/255.0,label

In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCHSIZE = 64

train_set = train_set.map(normalize_image, num_parallel_calls=AUTOTUNE)
train_set = train_set.cache()
train_set = train_set.shuffle(ds_info.splits["train"].num_examples)
train_set = train_set.batch(BATCHSIZE)
train_set = train_set.prefetch(AUTOTUNE)

test_set = test_set.map(normalize_image, num_parallel_calls=AUTOTUNE)
test_set = test_set.batch(BATCHSIZE)
test_set = test_set.prefetch(AUTOTUNE)

Make Model

In [5]:
class Relu(Layer):

    def __init__(self,):

        super(Relu, self).__init__()
    
    def call(self,x):
        
        return tf.math.maximum(x,0)
       
       
        
class Dense(Layer):

    def __init__(self,units):
        
        super(Dense, self).__init__()
        self.units = units

    
    def build(self,input_shape):
       
        self.w = self.add_weight(name="w",shape=(input_shape[-1],self.units),initializer="random_normal",trainable=True,)
        
        self.b = self.add_weight(name="b",shape=(self.units,),initializer="random_normal",trainable=True,)

    def call(self,inputs):
        return tf.matmul(inputs,self.w) + self.b

In [6]:
class myModel(Model):

    def __init__(self,number_of_classes = 10):

        super(myModel, self).__init__()

        self.dense1 = Dense(64)
        self.dense2 = Dense(32)
        self.classifier = Dense(number_of_classes)
        self.relu = Relu()
    
    def call(self,x):

        x = self.dense1(x)
   
        x = self.dense2(x)
        x = self.relu(x)
        x = self.classifier(x)
    
        return x


In [7]:
model = myModel()

model.compile(optimizer = "adam",loss=SparseCategoricalCrossentropy(from_logits=True),metrics="accuracy")

In [8]:
model.fit(train_set,epochs=10,verbose=2)

Epoch 1/10
938/938 - 11s - loss: 0.4590 - accuracy: 0.8688 - 11s/epoch - 12ms/step
Epoch 2/10
938/938 - 4s - loss: 0.2523 - accuracy: 0.9275 - 4s/epoch - 5ms/step
Epoch 3/10
938/938 - 4s - loss: 0.2118 - accuracy: 0.9379 - 4s/epoch - 5ms/step
Epoch 4/10
938/938 - 4s - loss: 0.1814 - accuracy: 0.9466 - 4s/epoch - 4ms/step
Epoch 5/10
938/938 - 4s - loss: 0.1590 - accuracy: 0.9522 - 4s/epoch - 4ms/step
Epoch 6/10
938/938 - 5s - loss: 0.1406 - accuracy: 0.9586 - 5s/epoch - 5ms/step
Epoch 7/10
938/938 - 6s - loss: 0.1265 - accuracy: 0.9612 - 6s/epoch - 6ms/step
Epoch 8/10
938/938 - 5s - loss: 0.1159 - accuracy: 0.9646 - 5s/epoch - 5ms/step
Epoch 9/10
938/938 - 5s - loss: 0.1075 - accuracy: 0.9671 - 5s/epoch - 5ms/step
Epoch 10/10
938/938 - 5s - loss: 0.1006 - accuracy: 0.9692 - 5s/epoch - 6ms/step


In [9]:
score = model.evaluate(test_set,verbose=2)
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

157/157 - 1s - loss: 0.1318 - accuracy: 0.9604 - 1s/epoch - 9ms/step
Test loss: 0.13176383078098297
Test accuracy: 0.9603999853134155
